# *Note on metric and hyperparameters*
##### Given that our data exihibits certain periodicity is reasonable to use 
##### a mean square error metric since it to penalizes more larger errors than smaller ones

##### The hyperparameters are picked based on
##### avoiding overfitting or underfitting.
##### This is attained by having a slightly higher score on the training data set respect to the test data set.
##### High train score and lower test score indicates overfitting.
##### High test score and lower train score indicates underfitting.



# *Note on the choosen interval:*
##### We test daily and montly predictions 
##### provided that daily data is significantly larger than monthly data
##### we can have a more appropiated modelling for such a escale.
##### If the data holds this kind of pattern for larger periods of time
##### we should expect the monthly analysis to be similar to the daily one.

# *Note on algorithms used*
##### we find below that a Linear Regression model seems
##### to work slightly better than Random Forests or XGBOOST algorithms
##### this could be due to the rather lack of complexity in our data and 
##### the way we treated it.
##### If we would be able to add more meaningful features to our model
##### that would make more likely to work better for those algorithms.

# *Check functions.py documentation for more info*

In [ ]:
# Going over all stations
from functions import *
stations =['AP001', 'AP005', 'AS001', 'BR005', 'BR006','BR007','BR008',
       'BR009', 'BR010', 'CH001', 'DL001', 'DL002', 'DL003', 'DL004',
       'DL005', 'DL006', 'DL007', 'DL008', 'DL009', 'DL010', 'DL011',
       'DL012', 'DL013', 'DL014', 'DL015', 'DL016', 'DL017', 'DL018',
       'DL019', 'DL020', 'DL021', 'DL022', 'DL023', 'DL024', 'DL025',
       'DL026', 'DL027', 'DL028', 'DL029', 'DL030', 'DL031', 'DL032',
       'DL033', 'DL034', 'DL035', 'DL036', 'DL037', 'DL038', 'GJ001',
       'HR011', 'HR012', 'HR013', 'HR014', 'JH001', 'KA002', 'KA003',
       'KA004', 'KA005', 'KA006', 'KA007', 'KA008', 'KA009', 'KA010',
       'KA011', 'KL002', 'KL004', 'KL007', 'KL008', 'MH005', 'MH006',
       'MH007', 'MH008', 'MH009', 'MH010', 'MH011', 'MH012', 'MH013',
       'MH014', 'ML001', 'MP001', 'MZ001', 'OD001', 'OD002', 'PB001',
       'RJ004', 'RJ005', 'RJ006', 'TG001', 'TG002', 'TG003', 'TG004',
       'TG005', 'TG006', 'TN001', 'TN002', 'TN003', 'TN004', 'TN005',
       'UP012', 'UP013', 'UP014', 'UP015', 'UP016', 'WB007', 'WB008',
       'WB009', 'WB010', 'WB011', 'WB012', 'WB013']



# **Day Prediction**

In [ ]:
#in order to get only the rows at 8 am is used: 
#!awk '/08:00:00/{print $0}' station_hour.csv > station_hour_8am.csv 
# The time is choosen arbitarily just to have data each 24 hours.
# (need to add the header: StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket)

file=['station_hour_8am.csv']
###HYPERPARAMETERS###
num_boost = [5,10,15,25] #in case of XGBOOST. It stands for number of boosting rounds in case of usinf xgboost
num_estim=[200,400,600] # in case of Random Forest. Number of estimators for random forest
past_observation_prepro=[20,40,70] #number of past observations to compute outliers
past_observations_model=[20,40,70] #number of past observations to use as predictors
smooth=[2,3,5,10] #smoothing signal factor
algor=['lr','random_for','xgboost'] # algorithm to be used
path_model=['./models/model_day']


#list with the r-squared metrics for train and test data
rs_train=[]
rs_test=[]
train_data_list=[]
test_pred_list=[]
for i in range(len(stations)):
    
    try:    
        rsquare=any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],
        name=stations[i],N=smooth[0],ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])

        rsquare_list=list(rsquare)
        rs_train.append(rsquare_list[0][0])
        rs_test.append(rsquare_list[0][1])
        train_data_list.append(rsquare_list[0][2])
        test_pred_list.append(rsquare_list[0][3])
    except IndexError:
        pass

print("mean of squared roots of rsquare_train")    
print(statistics.mean(np.sqrt(rs_train)))
print("median of squared roots of rsquare_train")    
print(statistics.median(np.sqrt(rs_train)))
print("mean of squared roots of rsquare_test")    
print(statistics.mean(np.sqrt(rs_test)))
print("median of squared roots of rsquare_test")    
print(statistics.median(np.sqrt(rs_test)))
# ERROR meaning the station data did not have enough data to take the past_observations lags
# or smoothing factor


print('#########\n############')

# We use the best configuration Found  (check further below)

'''
returns array with score difference. If the score is 0 the prediction is accurate.
If it's negative means that we predicted cleaner air quality than actually is. e.g. -1 could mean we predicted "good" air quality but in reality was just satisfactory.

If it's positive means that we predicted less clean air quality than actually is. e.g. 1 could mean we predicted "satisfactory" air quality but in reality was good.

The larger the number the larger we missed the right category (i.e. good,satifactpry, moderate, etc.)

On these grounds we will prefer always a positive difference over a negative one.
'''
print("For first station on the list:", stations[0])
precision=list(precision_array(num_past_observations_model=past_observations_model[0],N=smooth[0],test_data_AQI=train_data_list[0], test_pred_AQI=test_pred_list[0],day=True))
print("Air quality prediction\n",precision[0][0])
print('#####################\n#####################')
print("precision array:",precision[0][1])
print('#####################\n#####################')
print('Train and test plots for all the stations:')



In [ ]:
# We use the best configuration Found (check further below)
'''
returns array with score difference. If the score is 0 the prediction is accurate.
If it's negative means that we predicted cleaner air quality than actually is. e.g. -1 could mean we predicted "good" air quality but in reality was just satisfactory.

If it's positive means that we predicted less clean air quality than actually is. e.g. 1 could mean we predicted "satisfactory" air quality but in reality was good.

The larger the number the larger we missed the right category (i.e. good,satifactpry, moderate, etc.)

On these grounds we will prefer always a positive difference over a negative one.
'''
print("For first station on the list:", stations[0])
precision=list(precision_array(num_past_observations_model=past_observations_model[0],N=smooth[0],test_data_AQI=train_data_list[0], test_pred_AQI=test_pred_list[0]))
print("Air quality prediction\n",precision[0][0])
print('#####################\n#####################')
print("precision array:",precision[0][1])



# **Results for different Hyperparameters configurations**
##### For this could be more convinient a gridsearch for hyperparameter tunning, however for timewise reasons
##### and clarity we test manually some different configurations.

In [ ]:
"""
###HYPERPARAMETERS###
num_boost = [5,10,15,25] #in case of XGBOOST. It stands for number of boosting rounds in case of usinf xgboost
num_estim=[200,400,600] # in case of Random Forest. Number of estimators for random forest
past_observation_prepro=[20,40,70] #number of past observations to compute outliers
past_observations_model=[20,40,70] #number of past observations to use as predictors
smooth=[2,3,5,10] #smoothing signal factor
algor=['lr','random_for','xgboost'] # algorithm to be used
path_model=['./models/model_LR_v-1']


#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],name=stations[i],N=smooth[2],ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])
# To have a more intuitive result we print the square root of our metrics
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
9.945567740664915
9.746615186625352
43.00213386854126
44.02297692287345

#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[1],name=stations[i],N=smooth[2],ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])
# To have a more intuitive result we print the square root of our metrics
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
9.800340465716236
9.872106863987552
42.16095054763079
40.133623419359154

#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[2],name=stations[i],N=smooth[2],ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
9.376791406032845
9.525691364622407
40.115734445776795
40.19732814750828

#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],name=stations[i],N=smooth[3],ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
5.497086780625923
5.495244963003785
44.66817712849665
43.95780999458252

#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],name=stations[i],N=smooth[0],ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
23.315223846433774
22.684894016159095
39.519808018544516
39.294515129531895

#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],name=stations[i],N=1,ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
45.524304074925695
44.534034977083834
40.17374749052567
41.21088830132818

#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],name=stations[i],N=smooth[0],ml_algor=algor[1],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[1])
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
11.536505675357168
11.169206142780531
43.023179078687384
44.08070734781663



#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],name=stations[i],N=smooth[0],ml_algor=algor[1],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[1],boost_round=num_boost[1])
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
11.475091958011134
10.917182799036068
43.006824114037954
43.36383669884698

#RESULTS FOR THE CONFIGURATION:
any_station_day(general_file=file[0],num_past_observation_prepro=past_observation_prepro[0],num_past_observations_model=past_observations_model[0],name=stations[i],N=smooth[0],ml_algor=algor[2],path_store_model=path_model[0]+'_'+str(stations[i])+'.pkl',num_estim_rf=num_estim[1],boost_round=num_boost[1])
print(statistics.mean(np.sqrt(rs_train)))
print(statistics.median(np.sqrt(rs_train)))
print(statistics.mean(np.sqrt(rs_test)))
print(statistics.median(np.sqrt(rs_test)))
14.531872378601587
13.40621135839562
43.245711080827476
44.14326580666763
"""

# **Prediction for the next Month**

In [ ]:
# Going over all stations
from functions import *
stations =['AP001', 'AP005', 'AS001', 'BR005', 'BR006','BR007','BR008',
       'BR009', 'BR010', 'CH001', 'DL001', 'DL002', 'DL003', 'DL004',
       'DL005', 'DL006', 'DL007', 'DL008', 'DL009', 'DL010', 'DL011',
       'DL012', 'DL013', 'DL014', 'DL015', 'DL016', 'DL017', 'DL018',
       'DL019', 'DL020', 'DL021', 'DL022', 'DL023', 'DL024', 'DL025',
       'DL026', 'DL027', 'DL028', 'DL029', 'DL030', 'DL031', 'DL032',
       'DL033', 'DL034', 'DL035', 'DL036', 'DL037', 'DL038', 'GJ001',
       'HR011', 'HR012', 'HR013', 'HR014', 'JH001', 'KA002', 'KA003',
       'KA004', 'KA005', 'KA006', 'KA007', 'KA008', 'KA009', 'KA010',
       'KA011', 'KL002', 'KL004', 'KL007', 'KL008', 'MH005', 'MH006',
       'MH007', 'MH008', 'MH009', 'MH010', 'MH011', 'MH012', 'MH013',
       'MH014', 'ML001', 'MP001', 'MZ001', 'OD001', 'OD002', 'PB001',
       'RJ004', 'RJ005', 'RJ006', 'TG001', 'TG002', 'TG003', 'TG004',
       'TG005', 'TG006', 'TN001', 'TN002', 'TN003', 'TN004', 'TN005',
       'UP012', 'UP013', 'UP014', 'UP015', 'UP016', 'WB007', 'WB008',
       'WB009', 'WB010', 'WB011', 'WB012', 'WB013']



##### Inside the function any_station_month() there is a preprocessing done 
##### in order to get the AQI median value for each 15th day of a month.
##### The 15_th day is choosen arbitrarily just to have month-like data.

In [ ]:
#We use awk to get only the data corresponding to a 15th day
#awk '/-15/{print $0}' station_hour.csv > station_hour_15th.csv



file=['station_hour_15th.csv']
###HYPERPARAMETERS###
num_boost = [5,10,15,25] #in case of XGBOOST. It stands for number of boosting rounds in case of usinf xgboost
num_estim=[200,400,600] # in case of Random Forest. Number of estimators for random forest
past_observations_model=[3,5,10] #number of past observations to use as predictors
algor=['lr','random_for','xgboost'] # algorithm to be used
path_model=['./models/model_month']


#list with the r-squared metrics for train and test data
rs_train=[]
rs_test=[]
train_data_list=[]
test_pred_list=[]

for i in range(len(stations)):
    
    try:    
        rsquare=any_station_month(general_file=file[0],num_past_observations_model=past_observations_model[0],
        name=stations[i],ml_algor=algor[0],path_store_model=path_model[0]+'_'+str(stations[0])+'.pkl',num_estim_rf=num_estim[0],boost_round=num_boost[0])

        rsquare_list=list(rsquare)
        rs_train.append(rsquare_list[0][0])
        rs_test.append(rsquare_list[0][1])
        train_data_list.append(rsquare_list[0][2])
        test_pred_list.append(rsquare_list[0][3])
    except IndexError:
        pass
print("mean of squared roots of rsquare_train")    
print(statistics.mean(np.sqrt(rs_train)))
print("median of squared roots of rsquare_train")    
print(statistics.median(np.sqrt(rs_train)))
print("mean of squared roots of rsquare_test")    
print(statistics.mean(np.sqrt(rs_test)))
print("median of squared roots of rsquare_test")    
print(statistics.median(np.sqrt(rs_test)))



print("For first station on the list:", stations[0])
precision=list(precision_array(num_past_observations_model=past_observations_model[0],N=1,test_data_AQI=train_data_list[0], test_pred_AQI=test_pred_list[0],day=False))
print("Air quality prediction\n",precision[0][0])
print('#####################\n#####################')
print("precision array for the first station:",precision[0][1])
print('#####################\n#####################')
print('Train and test plots for all the stations:')
